# Intro

This notebook creates preprocesses the raw czech data from multiple .csv files, which can be obtained here https://data.world/lpetrocelli/some-translatedreformatted-czech-banking-data  

The raw csv files should be placed in the 'raw_czech' directory, and in pariticular, you need to have the files 'new_transaction.csv', 'new_disposition.csv' and 'new_client.csv'. If you wish to put the files in a different location, change the RAW_CSV_DIR variable below to the location of the raw files.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

import pickle
from datetime import date

In [2]:
RAW_CSV_DIR = "raw_czech"

for fname in os.listdir(RAW_CSV_DIR):
    if ".csv" in fname and "new_" in fname:
        print(fname)
        csv_name = fname.split(".")[0]
        if RAW_CSV_DIR[-1] == "/":
            RAW_CSV_DIR = RAW_CSV_DIR[:-1]
        
        cmd = f"{csv_name} = pd.read_csv('{RAW_CSV_DIR}/{fname}')"
        print("Running:", cmd)
        exec(cmd)


new_account.csv
Running: new_account = pd.read_csv('raw_czech/new_account.csv')
new_disposition.csv
Running: new_disposition = pd.read_csv('raw_czech/new_disposition.csv')
new_transaction.csv
Running: new_transaction = pd.read_csv('raw_czech/new_transaction.csv')
new_card.csv
Running: new_card = pd.read_csv('raw_czech/new_card.csv')
new_client.csv
Running: new_client = pd.read_csv('raw_czech/new_client.csv')


### Sort by acct 

In [3]:
tr_by_acct = new_transaction.sort_values(by = ["account_id", "date"])

In [13]:
df = pd.merge(
        pd.merge(
            tr_by_acct,
            new_disposition[new_disposition["type"]=="OWNER"][["account_id", "client_id"]],
            on="account_id",
        ),
        new_client[["client_id", "age"]],
        on = "client_id"
)
df

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol,client_id,age
0,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN,1,29
1,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN,1,29
2,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN,1,29
3,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED,1,29
4,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN,1,29
...,...,...,...,...,...,...,...,...,...,...
1056315,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN,13998,46
1056316,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN,13998,46
1056317,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN,13998,46
1056318,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED,13998,46


In [14]:
from datetime import datetime
czech_date_parser = lambda x: datetime.strptime(str(x), "%y%m%d")

df["datetime"] = df["date"].apply(czech_date_parser)
df

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol,client_id,age,datetime
0,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN,1,29,1995-03-24
1,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN,1,29,1995-04-13
2,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN,1,29,1995-04-23
3,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED,1,29,1995-04-30
4,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN,1,29,1995-05-13
...,...,...,...,...,...,...,...,...,...,...,...
1056315,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN,13998,46,1998-12-02
1056316,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN,13998,46,1998-12-10
1056317,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN,13998,46,1998-12-25
1056318,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED,13998,46,1998-12-31


In [15]:
df.to_csv("data/tr_by_acct_w_age.csv", index=False)